In [56]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup 
import random 
import multiprocessing
from tqdm import tqdm


df = pd.read_csv('./제주특별자치도음식점목록(통합).csv', sep=',', encoding='CP949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌

chromedriver = './chromedriver' 
driver = webdriver.Chrome(chromedriver) 

# driver = webdriver.Chrome(executable_path=r'C:\\Users\\ui\\Desktop\chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

In [57]:
# print( pd.__version__ )


In [58]:
df = df[df.영업상태명 != '폐업']

In [59]:
df = df.reset_index()

In [60]:
del df['index']

In [61]:
# df = df.iloc[9576:12768]
df = df.iloc[12768:15961]

In [62]:
# 문준영 12768~15961
df = df.reset_index()

In [63]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
12768,파파야,일반음식점,기타,20160412,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,32,제주특별자치도 서귀포시 성산읍 성산리 230-22번지,제주특별자치도 서귀포시 성산읍 성산등용로 7,63643.0,2018-08-31 23:59,
12769,청우식당,일반음식점,분식,20160414,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,93.5,제주특별자치도 서귀포시 강정동 165-7번지,"제주특별자치도 서귀포시 신서로48번길 20-9, 1층 (강정동)",63566.0,2020-05-14 2:40,
12770,이승악식당,일반음식점,한식,20160415,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,33,제주특별자치도 서귀포시 남원읍 신례리 2162-8번지,제주특별자치도 서귀포시 남원읍 서성로 308,63608.0,2019-04-24 2:40,
12771,당포회센타,일반음식점,회집,20160908,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,162.08,제주특별자치도 서귀포시 표선면 표선리 45-6번지,제주특별자치도 서귀포시 표선면 표선당포로 29,63629.0,2019-10-04 2:40,
12772,할매곱창,일반음식점,한식,20170317,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,115,제주특별자치도 서귀포시 동홍동 472-1번지,제주특별자치도 서귀포시 동홍중앙로66번길 22-10 (동홍동),63587.0,2018-08-31 23:59,
12773,해민이네양곱창,일반음식점,한식,20170320,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,100.75,제주특별자치도 서귀포시 법환동 737-3번지,"제주특별자치도 서귀포시 신중남로 26, 1층 (법환동)",63566.0,2018-08-31 23:59,
12774,농부의밥상,일반음식점,식육(숯불구이),20170322,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,84.9,제주특별자치도 서귀포시 성산읍 수산리 669-3,"제주특별자치도 서귀포시 성산읍 수시로10번길 18, 1층",63630.0,2021-04-17 2:40,
12775,리시온레스토랑,일반음식점,패밀리레스트랑,20170327,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,197.27,제주특별자치도 서귀포시 표선면 표선리 591-1번지,"제주특별자치도 서귀포시 표선면 표선동서로 263, 2층",63629.0,2018-08-31 23:59,
12776,팔미돈가2호점,일반음식점,한식,20170327,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,188.34,제주특별자치도 서귀포시 성산읍 난산리 2625-4,"제주특별자치도 서귀포시 성산읍 서성일로 511, 1층",63632.0,2021-06-20 2:40,
12777,키다리네,일반음식점,기타,20170328,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,66.4,제주특별자치도 서귀포시 안덕면 화순리 991-4번지 1층,"제주특별자치도 서귀포시 안덕면 화순임중로 53, 1층",63530.0,2019-04-11 2:40,


In [64]:
df = df[['사업장명', '업종구분대분류', '업종구분소분류', '소재지전체주소', 'naver_map_url']]
df

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,naver_map_url
12768,파파야,일반음식점,기타,제주특별자치도 서귀포시 성산읍 성산리 230-22번지,
12769,청우식당,일반음식점,분식,제주특별자치도 서귀포시 강정동 165-7번지,
12770,이승악식당,일반음식점,한식,제주특별자치도 서귀포시 남원읍 신례리 2162-8번지,
12771,당포회센타,일반음식점,회집,제주특별자치도 서귀포시 표선면 표선리 45-6번지,
12772,할매곱창,일반음식점,한식,제주특별자치도 서귀포시 동홍동 472-1번지,
12773,해민이네양곱창,일반음식점,한식,제주특별자치도 서귀포시 법환동 737-3번지,
12774,농부의밥상,일반음식점,식육(숯불구이),제주특별자치도 서귀포시 성산읍 수산리 669-3,
12775,리시온레스토랑,일반음식점,패밀리레스트랑,제주특별자치도 서귀포시 표선면 표선리 591-1번지,
12776,팔미돈가2호점,일반음식점,한식,제주특별자치도 서귀포시 성산읍 난산리 2625-4,
12777,키다리네,일반음식점,기타,제주특별자치도 서귀포시 안덕면 화순리 991-4번지 1층,


In [81]:
for i, keyword in enumerate(df['사업장명'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword)
    
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기
        
        time.sleep(2) # 중요함
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
        
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        
        print(final_url)
        df['naver_map_url'][i]=final_url
        
    except IndexError:
        df['naver_map_url'][i]= ''
        print('none')

In [77]:
# df
df['naver_map_url'].replace('', np.nan, inplace=True)

In [78]:
df = df.dropna(subset=['naver_map_url'])

In [79]:
df.to_csv('url_completed.csv', encoding = 'utf-8-sig')

In [80]:
df

,Unnamed: 0,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,naver_map_url


In [70]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re

from bs4 import BeautifulSoup
from tqdm import tqdm

# 웹드라이버 접속
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

df = pd.read_csv('url_completed.csv') #null값 처리 완료 데이터

# 수집할 정보들
rating_list = [] # 평점
user_review_id = {} # 유저의 id - 추천시스템에서는 필수
review_json = {} # 리뷰
image_json = {} # 이미지

In [71]:
for i in range(len(df)):
    
    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            time.sleep(1)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(3)
            driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
            time.sleep(3)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['사업장명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = ""
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = ""
            print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})

            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                
                try:
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except :
                    if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                    elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0):
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                    else :
                        img_url = ""
            else:
                img_url = ""
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

In [72]:
rating_df = pd.DataFrame(rating_list)
rating_df.columns = ['UserID','ItemID','Rating','Timestamp']
rating_df.to_csv('rating9.csv', encoding='utf-8-sig')

import json

file_path = "./user_review_id.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(user_review_id, outfile, ensure_ascii=False)
    
file_path = "./review.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(review_json, outfile, ensure_ascii=False)

file_path = "./image.json"
with open(file_path,'w', encoding="UTF-8") as outfile:
    json.dump(image_json, outfile, ensure_ascii=False)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

## 최종 크롤링된 데이터프레임에
## 공통 값을 기준으로(열 기준) concat하여 주소 붙이기 !!!